In [1]:
!pip install pandas
!pip install sodapy
!pip install pymssql


In [2]:
import pymssql
from getpass import getpass

In [4]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.kingcounty.gov", None)

#Uvg3EEyR1TnfEqwO6UMA3NJbS

MyAppToken = getpass()

client = Socrata("data.kingcounty.gov",
                 MyAppToken,
                  username="dongsoomikeseo@gmail.com",
                 password=getpass())

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("xmnh-jvpd", limit = 509000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

········
········


In [6]:
results_df

,precinct,race,leg,cc,cg,countergroup,party,countertype,sumofcount
0,SHL 32-0001,City of Shoreline Council Position No. 2,32,1,7,Total,NP,Registered Voters,544
1,SHL 32-0001,City of Shoreline Council Position No. 2,32,1,7,Total,NP,Times Counted,178
2,SHL 32-0001,City of Shoreline Council Position No. 2,32,1,7,Total,NP,Times Under Voted,26
3,SHL 32-0001,City of Shoreline Council Position No. 2,32,1,7,Total,<NP>,Vivian Collica,53
4,SHL 32-0001,City of Shoreline Council Position No. 2,32,1,7,Total,<NP>,Keith Patrick Scully,99
...,...,...,...,...,...,...,...,...,...
508716,ELECTIONS OFFICE,King County Advisory Vote No. 28,0,0,9,Total,NP,Times Counted,2
508717,ELECTIONS OFFICE,King County Advisory Vote No. 28,0,0,9,Total,NP,Times Under Voted,0
508718,ELECTIONS OFFICE,King County Advisory Vote No. 28,0,0,9,Total,<NP>,Repealed,0
508719,ELECTIONS OFFICE,King County Advisory Vote No. 28,0,0,9,Total,<NP>,Maintained,2


In [19]:
only976 =results_df.loc[(((results_df['race'] == 'King County Initiative Measure No. 976')&(results_df['countertype'] == 'Yes'))|\
                ((results_df['race'] == 'King County Initiative Measure No. 976')&(results_df['countertype'] == 'No')))]
only976

,precinct,race,leg,cc,cg,countergroup,party,countertype,sumofcount
35,SHL 32-0001,King County Initiative Measure No. 976,32,1,7,Total,<NP>,Yes,63
36,SHL 32-0001,King County Initiative Measure No. 976,32,1,7,Total,<NP>,No,107
235,LFP 46-0003,King County Initiative Measure No. 976,46,1,7,Total,<NP>,Yes,103
236,LFP 46-0003,King County Initiative Measure No. 976,46,1,7,Total,<NP>,No,157
450,BUR 33-0009,King County Initiative Measure No. 976,33,8,7,Total,<NP>,Yes,64
...,...,...,...,...,...,...,...,...,...
508206,AUB 47-3812,King County Initiative Measure No. 976,47,7,8,Total,<NP>,No,27
508400,RNT 05-3813,King County Initiative Measure No. 976,5,9,8,Total,<NP>,Yes,2
508401,RNT 05-3813,King County Initiative Measure No. 976,5,9,8,Total,<NP>,No,5
508602,ELECTIONS OFFICE,King County Initiative Measure No. 976,0,0,9,Total,<NP>,Yes,0


In [32]:
only976YesNoSum =only976[['precinct','race','sumofcount']].groupby(['precinct','race']).sum()

only976YesNoSum_final =only976YesNoSum.reset_index()

only976YesNoSum_final

,precinct,race,sumofcount
0,ADAIR,King County Initiative Measure No. 976,166221
1,ALDARRA,King County Initiative Measure No. 976,216204
2,ALDER SPRINGS,King County Initiative Measure No. 976,167103
3,ALDERWOOD,King County Initiative Measure No. 976,202275
4,ALG 30-0013,King County Initiative Measure No. 976,12833
...,...,...,...
2607,WOODSIDE,King County Initiative Measure No. 976,127121
2608,WYNOCHE,King County Initiative Measure No. 976,13073
2609,WYNONA,King County Initiative Measure No. 976,89141
2610,YPT 48-1233,King County Initiative Measure No. 976,94110


In [40]:
only976no = only976.loc[only976['countertype'] =='No']
only976no =only976no[['precinct','countertype','sumofcount']]
only976merged = only976YesNoSum_final.merge(only976no, on='precinct')
only976merged


,precinct,race,sumofcount_x,countertype,sumofcount_y
0,ADAIR,King County Initiative Measure No. 976,166221,No,221
1,ALDARRA,King County Initiative Measure No. 976,216204,No,204
2,ALDER SPRINGS,King County Initiative Measure No. 976,167103,No,103
3,ALDERWOOD,King County Initiative Measure No. 976,202275,No,275
4,ALG 30-0013,King County Initiative Measure No. 976,12833,No,33
...,...,...,...,...,...
2607,WOODSIDE,King County Initiative Measure No. 976,127121,No,121
2608,WYNOCHE,King County Initiative Measure No. 976,13073,No,73
2609,WYNONA,King County Initiative Measure No. 976,89141,No,141
2610,YPT 48-1233,King County Initiative Measure No. 976,94110,No,110
